In [1]:
import pandas as pd

In [2]:
df_sentiment=pd.read_csv('imdb_labelled.txt',sep='\t',names=['Comment','Label'])

In [3]:
df_sentiment.head()

,Comment,Label
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [4]:
df_sentiment.describe()

,Label
count,748.000000
mean,0.516043
std,0.500077
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


In [5]:
df_sentiment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 748 entries, 0 to 747
Data columns (total 2 columns):
Comment    748 non-null object
Label      748 non-null int64
dtypes: int64(1), object(1)
memory usage: 11.8+ KB


In [6]:
df_sentiment.groupby('Label').describe()

Comment                                              
        count unique                               top freq
Label                                                      
0         362    361                Not recommended.      2
1         386    384  Definitely worth checking out.      2

In [7]:
df_sentiment['Length']=df_sentiment['Comment'].apply(len)

In [8]:
df_sentiment.head()

,Comment,Label,Length
0,"A very, very, very slow-moving, aimless movie ...",0,87
1,Not sure who was more lost - the flat characte...,0,99
2,Attempting artiness with black & white and cle...,0,188
3,Very little music or anything to speak of.,0,44
4,The best scene in the movie was when Gerardo i...,1,108


In [9]:
df_sentiment[df_sentiment['Length'] > 50]['Comment'].iloc[0]

'A very, very, very slow-moving, aimless movie about a distressed, drifting young man.  '

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer=CountVectorizer()

In [14]:
import string
from nltk.corpus import stopwords
#define a function to get rid of stopwords present in the messages
def message_text_process(mess):
    no_punchuation=[char for char in mess if char not in string.punctuation]
    no_punchuation=''.join(no_punchuation)
    return [word for word in no_punchuation.split() if word.lower() not in stopwords.words('english')]
df_sentiment['Comment'].head(5).apply(message_text_process)

0    [slowmoving, aimless, movie, distressed, drift...
1    [sure, lost, flat, characters, audience, nearl...
2    [Attempting, artiness, black, white, clever, c...
3                     [little, music, anything, speak]
4    [best, scene, movie, Gerardo, trying, find, so...
Name: Comment, dtype: object

In [16]:
bagofwords_tr=CountVectorizer(analyzer=message_text_process).fit(df_sentiment['Comment'])
Comment_bagofwords=bagofwords_tr.transform(df_sentiment['Comment'])

In [17]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer=TfidfTransformer().fit(Comment_bagofwords)
comment_tfidf=tfidf_transformer.transform(Comment_bagofwords)
print(comment_tfidf.shape)

(748, 3259)


In [18]:
from sklearn.naive_bayes import MultinomialNB
spam_detection_model=MultinomialNB().fit(comment_tfidf,df_sentiment['Label'])
To_predict_Comment=df_sentiment['Comment'][0]
Comment_bagofwords=bagofwords_tr.transform([To_predict_Comment])
tkidf=tfidf_transformer.transform(Comment_bagofwords)
print("predicted:",spam_detection_model.predict(tkidf)[0])
print("Expected:",df_sentiment['Label'][3])

predicted: 0
Expected: 0
